# Test Version Tracking Scenarios

This notebook demonstrates how to test the version tracking pipeline with the generated test data.

## Test Data Overview

The test data includes:
- **5 markdown files** with actual content
- **4 daily snapshots** showing timeline of changes
- **3 scenarios** covering all version tracking cases

## Scenarios

1. **Scenario 1 (Day 2):** Modify existing page → version +1 for that page only
2. **Scenario 2 (Day 3):** Insert new page → subsequent pages shift +1, versions +1
3. **Scenario 3 (Day 4):** Delete page → subsequent pages shift -1, versions +1

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path

# Database path
TEST_DB = Path('../databases/content_repo_test.db')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Scenario 1: Modify Existing Page

**What happens:** Page 2 content is modified (sick leave policy updated)

**Expected behavior:**
- Page 2: version 1 → 2, checksum changes
- Page 1: unchanged
- Page 3: unchanged

In [2]:
conn = sqlite3.connect(TEST_DB)

# Day 1 state
day1 = pd.read_sql("""
    SELECT ud_source_file_id, raw_file_page_nbr, version_nbr, 
           substr(content_checksum, 1, 16) as checksum_short
    FROM content_repo_day1
    ORDER BY raw_file_page_nbr
""", conn)

print("DAY 1 - Initial State:")
print(day1)

# Day 2 state
day2 = pd.read_sql("""
    SELECT ud_source_file_id, raw_file_page_nbr, version_nbr,
           substr(content_checksum, 1, 16) as checksum_short
    FROM content_repo_day2
    ORDER BY raw_file_page_nbr
""", conn)

print("\nDAY 2 - After Modifying Page 2:")
print(day2)

# Compare
print("\n📊 Changes from Day 1 to Day 2:")
for idx in range(len(day1)):
    old = day1.iloc[idx]
    new = day2.iloc[idx]
    if old['version_nbr'] != new['version_nbr'] or old['checksum_short'] != new['checksum_short']:
        print(f"  ✏️ Page {old['raw_file_page_nbr']}: v{old['version_nbr']}→v{new['version_nbr']}, checksum changed")
    else:
        print(f"  ✅ Page {old['raw_file_page_nbr']}: unchanged")

conn.close()

DAY 1 - Initial State:
   ud_source_file_id  raw_file_page_nbr  version_nbr    checksum_short
0               1001                  1            1  620c75e45bb1c3a1
1               1002                  2            1  b04ed99ce1bf45e4
2               1003                  3            1  1e4bd8f57457cee3

DAY 2 - After Modifying Page 2:
   ud_source_file_id  raw_file_page_nbr  version_nbr    checksum_short
0               1001                  1            1  620c75e45bb1c3a1
1               1002                  2            2  ce9d64d784d0d261
2               1003                  3            1  1e4bd8f57457cee3

📊 Changes from Day 1 to Day 2:
  ✅ Page 1: unchanged
  ✏️ Page 2: v1→v2, checksum changed
  ✅ Page 3: unchanged


## View Actual Content Changes

Let's see what actually changed in the markdown content:

In [ ]:
# Read the modified page versions
with open('../test_data_version_tracking/markdown_files/leave_policy/page2_v1.md', 'r') as f:
    content_v1 = f.read()

with open('../test_data_version_tracking/markdown_files/leave_policy/page2_v2.md', 'r') as f:
    content_v2 = f.read()

print("PAGE 2 - Version 1:")
print("="*60)
print(content_v1)
print("\nPAGE 2 - Version 2 (UPDATED):")
print("="*60)
print(content_v2)

## Scenario 2: Insert New Page

**What happens:** New page inserted at position 3

**Expected behavior:**
- New page (id=1004) appears at position 3, version 1
- Old page 3 (id=1003) shifts to position 4, version 1→2
- Pages 1-2: unchanged

In [ ]:
conn = sqlite3.connect(TEST_DB)

# Day 2 state
day2 = pd.read_sql("""
    SELECT ud_source_file_id, raw_file_page_nbr, version_nbr,
           substr(content_checksum, 1, 16) as checksum_short
    FROM content_repo_day2
    ORDER BY raw_file_page_nbr
""", conn)

print("DAY 2 - Before Insertion:")
print(day2)

# Day 3 state
day3 = pd.read_sql("""
    SELECT ud_source_file_id, raw_file_page_nbr, version_nbr,
           substr(content_checksum, 1, 16) as checksum_short
    FROM content_repo_day3
    ORDER BY raw_file_page_nbr
""", conn)

print("\nDAY 3 - After Inserting New Page 3:")
print(day3)

print("\n📊 Changes from Day 2 to Day 3:")

# Check for new pages
new_ids = set(day3['ud_source_file_id']) - set(day2['ud_source_file_id'])
if new_ids:
    for new_id in new_ids:
        new_page = day3[day3['ud_source_file_id'] == new_id].iloc[0]
        print(f"  🆕 NEW: content_id={new_id} at page {new_page['raw_file_page_nbr']}, v{new_page['version_nbr']}")

# Check for shifted pages
for _, row in day2.iterrows():
    old_page = row['raw_file_page_nbr']
    content_id = row['ud_source_file_id']
    
    new_row = day3[day3['ud_source_file_id'] == content_id]
    if len(new_row) > 0:
        new_row = new_row.iloc[0]
        new_page = new_row['raw_file_page_nbr']
        
        if old_page != new_page:
            print(f"  📍 SHIFTED: content_id={content_id}, page {old_page}→{new_page}, v{row['version_nbr']}→v{new_row['version_nbr']}")
        elif row['version_nbr'] == new_row['version_nbr']:
            print(f"  ✅ UNCHANGED: content_id={content_id}, page {old_page}, v{row['version_nbr']}")

conn.close()

## Scenario 3: Delete Page

**What happens:** Page 2 (id=1002) is deleted

**Expected behavior:**
- Page 2 (id=1002) marked Inactive
- Old page 3 (id=1004) shifts to position 2, version 1→2
- Old page 4 (id=1003) shifts to position 3, version 2→3
- Page 1: unchanged

In [ ]:
conn = sqlite3.connect(TEST_DB)

# Day 3 state
day3 = pd.read_sql("""
    SELECT ud_source_file_id, raw_file_page_nbr, version_nbr,
           substr(content_checksum, 1, 16) as checksum_short, file_status
    FROM content_repo_day3
    ORDER BY raw_file_page_nbr
""", conn)

print("DAY 3 - Before Deletion:")
print(day3)

# Day 4 state
day4 = pd.read_sql("""
    SELECT ud_source_file_id, raw_file_page_nbr, version_nbr,
           substr(content_checksum, 1, 16) as checksum_short, file_status
    FROM content_repo_day4
    ORDER BY CASE WHEN file_status = 'Active' THEN 0 ELSE 1 END, raw_file_page_nbr
""", conn)

print("\nDAY 4 - After Deleting Page 2:")
print(day4)

print("\n📊 Changes from Day 3 to Day 4:")

# Check for deleted pages
for _, row in day3.iterrows():
    content_id = row['ud_source_file_id']
    old_page = row['raw_file_page_nbr']
    
    new_row = day4[day4['ud_source_file_id'] == content_id]
    if len(new_row) > 0:
        new_row = new_row.iloc[0]
        
        if new_row['file_status'] == 'Inactive' and row['file_status'] == 'Active':
            print(f"  🗑️ DELETED: content_id={content_id}, page {old_page}, marked Inactive")
        elif old_page != new_row['raw_file_page_nbr']:
            print(f"  📍 SHIFTED: content_id={content_id}, page {old_page}→{new_row['raw_file_page_nbr']}, v{row['version_nbr']}→v{new_row['version_nbr']}")
        elif row['version_nbr'] == new_row['version_nbr']:
            print(f"  ✅ UNCHANGED: content_id={content_id}, page {old_page}, v{row['version_nbr']}")

conn.close()

## Test Pipeline with Day 1 → Day 2 Transition

This shows how to use the test data with your version tracking pipeline:

In [ ]:
import hashlib

def calculate_file_checksum(file_path):
    """Calculate SHA-256 checksum from file"""
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            return hashlib.sha256(content).hexdigest()
    except FileNotFoundError:
        return None

conn = sqlite3.connect(TEST_DB)

# Simulate pipeline: Load Day 1 as "existing version history"
existing_versions = pd.read_sql("""
    SELECT 
        ud_source_file_id as content_id,
        raw_file_nme as file_name,
        raw_file_page_nbr as page_number,
        version_nbr as version_sequence,
        content_checksum,
        extracted_markdown_file_path
    FROM content_repo_day1
""", conn)

print("Existing Version History (Day 1):")
print(existing_versions[['content_id', 'page_number', 'version_sequence', 'content_checksum']].head())

# Load Day 2 as "new content"
new_content = pd.read_sql("""
    SELECT 
        ud_source_file_id,
        raw_file_nme,
        raw_file_page_nbr,
        content_checksum,
        extracted_markdown_file_path,
        last_modified_dt
    FROM content_repo_day2
""", conn)

# Calculate checksums from actual files (as pipeline does)
for idx, row in new_content.iterrows():
    calculated_checksum = calculate_file_checksum(row['extracted_markdown_file_path'])
    new_content.at[idx, 'calculated_checksum'] = calculated_checksum

print("\nNew Content (Day 2):")
print(new_content[['ud_source_file_id', 'raw_file_page_nbr', 'content_checksum', 'calculated_checksum']].head())

# Detect changes
print("\n🔍 Change Detection:")
for _, content in new_content.iterrows():
    content_id = content['ud_source_file_id']
    page_num = content['raw_file_page_nbr']
    file_name = content['raw_file_nme']
    new_checksum = content['calculated_checksum']
    
    # Find existing version
    existing = existing_versions[
        (existing_versions['file_name'] == file_name) & 
        (existing_versions['page_number'] == page_num)
    ]
    
    if len(existing) == 0:
        print(f"  🆕 NEW PAGE: content_id={content_id}, page {page_num}")
    else:
        old_checksum = existing.iloc[0]['content_checksum']
        old_version = existing.iloc[0]['version_sequence']
        
        if old_checksum != new_checksum:
            print(f"  ✏️ MODIFIED: content_id={content_id}, page {page_num}, v{old_version}→v{old_version+1}")
            print(f"     Old checksum: {old_checksum[:32]}...")
            print(f"     New checksum: {new_checksum[:32]}...")
        else:
            print(f"  ✅ UNCHANGED: content_id={content_id}, page {page_num}, v{old_version}")

conn.close()

## Summary

This test data provides:

1. **Realistic scenarios** for all 3 version tracking cases
2. **Actual markdown files** for checksum calculation
3. **Daily snapshots** for incremental testing
4. **Expected outcomes** for validation

## Next Steps

1. Test each scenario individually
2. Verify version history records are created correctly
3. Check FAQ deprecation logic
4. Validate page number shift detection